In [ ]:
# !pip install control # Activar si corre en Colab, 1 vez por sesión.

# Sección de librerías Python

In [ ]:
import numpy as np
import scipy.linalg
from scipy import signal
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import control as ctl
from control.matlab import *
from sympy import *

# Modelo matemático

El modelo matemático nos permite llevar a cabo cálculos de ganancias para el controlador ___Kp___, así como analizar su comportamiento de acuerdo a cambios en parámetros del mismo.

Primero introduzcamos la planta con la cual estaremos trabajando:

In [ ]:
time = np.linspace(0,50,10000)  # Ejemplo genérico, substituir por el sistema modelado.
s = TransferFunction.s
o_l = 1/(s**3 + 9*s**2 + 8*s) 
nvo = tf(o_l)

# Para calcular el límite de las constantes de error, es necesario obtener límites,
# la librería para obtener los límites requiere que la variable sea "x".
x = symbols('x')
FT_xs1_1 = 1/(x**3 + 6*x**2 + 8*x)
FT_xs2_1 = x/(x**3 + 6*x**2 + 8*x)
FT_xs3_1 = x**2/(x**3 + 6*x**2 + 8*x)

########## Impresión del límite de la función.    
print("Función de transferencia: {}".format(nvo))  
      
print("Límite de la FT conforme x --> 0 : {}".format(limit(FT_xs1_1, x, 0)))
print("Límite de la FT conforme x --> 0 : {}".format(limit(FT_xs2_1, x, 0)))
print("Límite de la FT conforme x --> 0 : {}".format(limit(FT_xs3_1, x, 0)))


print("Ess A/s Sistema tipo 1: {}".format(1./(1 + limit(FT_xs1_1, x, 0))))
print("Ess A/s^2 Sistema tipo 1: {}".format(1./(limit(FT_xs2_1, x, 0))))
print("Ess A/s^3 Sistema tipo 1: {}".format(1./(limit(FT_xs3_1, x, 0))))

# Análisis ESS
## Step function
fb_nvo = feedback(nvo,sign = -1)
X0 = 0 # Utilizaremos las mismas condiciones iniciales para todas las simulaciones.

t_1 = np.linspace(0,10,10000) # Vamos a utilizar la misma escala y división para todas las simulaciones.

y_nvo_s, t_nvo_s = ctl.matlab.step(fb_nvo, t_1, X0)

## Ramp function
m = 13.33
u_ramp = m*time
t_nvo_r, y_nvo_r = ctl.forced_response(fb_nvo, t_1, u_ramp)

## Parabolic function
r = 0.5
u_parabola = r*time**2
t_nvo_p, y_nvo_p= ctl.forced_response(fb_nvo, t_1, u_parabola)

# Ploteo de las señales:
plt.subplot(311)
plt.plot(t_nvo_s, y_nvo_s)
plt.grid(True)
plt.ylabel('Escalón')

plt.subplot(312)
plt.plot(t_nvo_r, y_nvo_r)
plt.plot(t_nvo_r, u_ramp) #
plt.grid(True)
plt.ylabel('Rampa')

plt.subplot(313)
plt.plot(t_nvo_p, y_nvo_p)
plt.plot(t_nvo_p, u_parabola) #
plt.grid(True)
plt.ylabel('Parábola')

plt.tight_layout()
plt.show()

# Gráfica del lugar de las raíces

De acuerdo al régimen de operación solicitado, calcule el valor de la ganancia K a partir del diagrama de lugar de las raíces.

In [ ]:
ctl.rlocus(nvo);

# Gráfica de Bode del sistema, sólo si se requiere en el diseño

In [ ]:
ctl.bode_plot(nvo);

# Diseño para placa Arduino (sistema discretizado)

Cálculo de constantes de control (Kp) para el sistema discreto, el valor obtenido en este cálculo se substituye en el archivo *.ino para correr en la placa de Arduino.

In [ ]:
T = 50/1000 # Valor de la velocidad de muestreo de la placa Arduino
Zc=exp(T*Pc)